# Estimating Snow Depth from ICESat-2 ATL08QL data

This script is made to derive preliminary snow depth estimates using the ATL08 Quick Look product. Because the quick looks are (a) subject to geolocation errors and (b) automatically deleted after the final data release, it is recommended to use `is2_snowex_ak.ipynb` if it has been ~45 days since the RGT of interest.

icepyx normally does not support downloading of ATL08QL data, so a slightly altered version is required. The user may find a branch with ATL08QL functionality at: https://github.com/zachghiaccio/icepyx/tree/quick-looks. A pull request for this feature will be introduced to the icepyx development team in the near future.

When ATL08QL functionality is introduced to icepyx, it is planned to integrate this notebook with `is2_snowex_ak.ipynb`.

As with the above notebook, snow-on/-off DEMs from the UAF airborne lidar are needed to run this script. Interested users should contact the original provider of the data (Chris Larsen, cflarsen@uaf.edu).

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import rioxarray as rio
import os
from osgeo import gdal
import icepyx as ipx
import pandas as pd
import pickle
from pyproj import Proj, transform, Transformer, CRS
import sys

sys.path.append('C:/Users/zfair/OneDrive - NASA/Documents/Python/icesat2-snow/')
import lidar_processing as lp

To keep things simple, we are not using cloud services to obtain the ATL08QL data - the data will be downloaded to our workspace. Once this notebook is integrated, the hope is to allow cloud access to quick look data.

In [ ]:
# Specify the ICESat-2 product
short_name = 'ATL08QL'

# Define the spatial extent using a pre-generated bounding box
with open('C:/Users/zfair/OneDrive - NASA/Documents/Python/icesat2-snow/Data/shapefiles/snowex_sites_for_icepyx.pkl', 'rb') as f:
    coordinates = pickle.load(f)
    spatial_extent = coordinates['alaska']
    
# Date range of requested data
date_range = ['2023-03-01', '2023-03-31']

# ICESat-2 track (RGT, optional)
rgt = '1356'
    
# Generate the query object
try:
    # With RGT specification
    region = ipx.Query(short_name, spatial_extent, date_range, tracks=rgt)
except:
    # General query
    region = ipx.Query(short_name, spatial_extent, date_range)
    
print(region)

In [ ]:
# Enter login information for Earthdata
uid = 'zhfair'
email = 'zhfair@umich.edu'
region.earthdata_login(uid, email)

In [ ]:
# Check the ids for the requested data
region.avail_granules(ids=True)

date_str = region.avail_granules(ids=True)[0][0][8:16]
print(date_str)

In [ ]:
# Start the data order
region.order_granules()

In [ ]:
# Download the granules into the specified folder
path = 'C:/Users/zfair/OneDrive - NASA/Documents/Python/icesat2-snow/Data/is2/icepyx/'
region.download_granules(path)

Now that our data is downloaded, let's load the file into a DataFrame.

In [ ]:
# Find file in saved directory
for fname in os.listdir(path):
    if rgt and date_str in fname:
        is2_file = path + fname
                    
print(is2_file)
                    
# Load the ATL08QL data into a DataFrame
with h5py.File(is2_file, 'r') as f:
    # Identify the strong beams
    sc_orient = f['orbit_info/sc_orient'][0]
    strong_beams, strong_ids = lp.strong_beam_finder(sc_orient)
    
    # Concatenate strong beam data into DataFrame
    atl08 = lp.beam_cycle_concat([is2_file], 'ATL08')
    
atl08.head()

In [ ]:
# Add easting/northing coordinates (EPSG:32606)
transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32606', always_xy=True)

atl08['x'], atl08['y'] = transformer.transform(atl08.lon, atl08.lat)

atl08.head()

In [ ]:
# Remove messy ICESat-2 data
upper = atl08.height.mean() + 3*atl08.height.std()
atl08 = atl08.loc[atl08.height<upper]

# Add vertical height correction needed for quick-look products
atl08['height'] += 2.7

Our ICESat-2 data is now ready! Let's load up a snow-off DEM from the UAF airborne lidar.

In [ ]:
# Load UAF lidar data into a raster. Will be updated to be more flexible at a later date.
f_snow_off = 'C:/Users/zfair/OneDrive - NASA/Documents/Python/icesat2-snow/Data/UAF/farmersloop_2022may28_dtm.tif'
f_snow_on = 'C:/Users/zfair/OneDrive - NASA/Documents/Python/icesat2-snow/Data/UAF/farmersloop_2022oct24_snowdepth.tif'

lidar_snow_off = rio.open_rasterio(f_snow_off)
lidar_snow_on = rio.open_rasterio(f_snow_on)

lidar_snow_off

In [ ]:
# Apply vertical datum correction. To be updated to be more flexible.
lidar_snow_off -= 9.75

# Coregister with ICESat-2 using a bivariate spline
strong_ids = np.unique(atl08['gt'])
atl08_uaf = lp.coregister_is2(lidar_snow_off, lidar_snow_on, atl08, strong_ids)

atl08_uaf.head()

In [ ]:
# Remove filler data
atl08_uaf['lidar_height'][atl08_uaf['lidar_height']<0] = np.nan
atl08_uaf['lidar_snow_depth'][atl08_uaf['lidar_snow_depth']<0] = np.nan
atl08_uaf['residual'][atl08_uaf['residual']>100] = np.nan

atl08_uaf.head()

In [ ]:
atl08_uaf.plot.scatter(x='y', y='residual')